In [2]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd

d:\mamba\envs\sww-prod\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
df = pd.read_excel("data\Initial_data.xlsx")

In [14]:
template = "All the materials were synthesized by the co-precipitation technique. First, 1 M CaCl2 with final concentration of Ca ion being {} mM was mixed with {} polymer with molecular weight of {} kDa and content being {} % wt. Then, {} solvent was added with final volume content being {}, following adjustment with distilled water up to 500 mkl. Then, 0.1 M Na2CO3 with final concentration of CO3 ions being {} was mixed with 0.1 M of NaHCO3 with final concentration of HCO3 ions being {} and {} surfactant with content being {} % wt. Then, same solvent was added, following adjustment with distilled water up to 500 mkl. Two resulting solutions, heated up to {} C before the reaction, were mixed under the stirring with {} rpm, while the temperature kept unchanged. Reaction proceeded for {} sec following centrifugation."

In [30]:
def time_to_sec(time):
     min, sec = (time.split()[0], time.split()[2])
     seconds = int(min) * 60 + int(sec)
     return seconds

In [31]:
df['Synthesis time'] = df['Synthesis time'].apply(time_to_sec)

In [32]:
def format_text(row):
    return template.format(row['Ca ion, mM'], row['Polymer type'], row['Polymer Mwt, kDa'], row['Polymer, % wt.'], row['Solvent type'], row['Solvent, % vol.'], row['CO3 ion, mM'], row['HCO3 ion, mM'], row['Surfactant type'], row['Surfactant, % wt.'], row['Temperature, C'], row['Stirring, rpm'], row['Synthesis time'])

text_list = df.apply(format_text, axis=1)

In [33]:
encoded_inputs = list(map(lambda t: tokenizer(t, return_tensors='pt', padding='max_length', max_length=250), text_list))

In [34]:
with torch.no_grad():
    outputs = torch.cat([model(**input_dict).last_hidden_state for input_dict in encoded_inputs])

In [22]:
outputs.shape

torch.Size([215, 250, 768])